In [54]:
import sys, os, csv, lucene

In [2]:
from java.io import File
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene.index import DirectoryReader
from org.apache.lucene.queryparser.classic import QueryParser
from org.apache.lucene.search import IndexSearcher
from org.apache.lucene.store import SimpleFSDirectory
from org.apache.lucene.util import Version

In [3]:
lucene.initVM(vmargs=['-Djava.awt.headless=true'])

In [4]:
fsDir = SimpleFSDirectory(File("index"))
searcher = IndexSearcher(DirectoryReader.open(fsDir))

In [18]:
analyzer = StandardAnalyzer(Version.LUCENE_CURRENT)
parser = QueryParser(Version.LUCENE_CURRENT, "question", analyzer)

In [47]:
query = parser.parse('question:When athletes begin to exercise, their heart rates and respiration rates increase.  At what level of organization does the human body coordinate these functions? answer:at the cellular level')

In [52]:
result = searcher.search(query, 10)
print result.getMaxScore()
scoreDocs = result.scoreDocs

5.66312217712


In [49]:
for scoreDoc in scoreDocs:
    doc = searcher.doc(scoreDoc.doc)
    print [doc.get("question"), doc.get("answer"), searcher.explain(query, scoreDoc.doc).getValue()]

[u'When athletes begin to exercise, their heart rates and respiration rates increase. At what level of organization does the human body coordinate these functions?', u'at the system level', 5.66312313079834]
[u'at what level of organization does life begin', u'cell', 0.9043471813201904]
[u'At what level of organization does life begin?', u'Cells', 0.9043471813201904]
[u'Level of Organization in the Human Body', u'Cell > Tissue > Organs > Organ Systems> Organisms', 0.8805510997772217]
[u'What are in the Cellular level organization', u'Atoms, Molecules, Organelles, Cells', 0.8511862754821777]
[u'What are in the Cellular level organization', u'Atoms, Molecules, Organelles, Cells', 0.8511862754821777]
[u'What is the largest level of organization in the human body?', u'the largest level of organization in the human body is the organ system.', 0.8304603695869446]
[u'What level in the human body would be the heart?', u'It would be a third level organ (p.510)', 0.7424818873405457]
[u'the pheno

In [73]:
with open("../data/training_set.tsv") as f:
    reader = csv.reader(f, delimiter="\t", strict=True, quoting=csv.QUOTE_NONE)
    header = next(reader)  # ignore header
    is_train_set = (len(header) == 7)
    lines = list(reader)

In [74]:
def score_qa(q, a):
    query = parser.parse('question:(%s) answer:(%s)' % (QueryParser.escape(q), QueryParser.escape(a)))
    result = searcher.search(query, 1)
    return result.getMaxScore()    

correct = 0
preds = []
scores = []
for id, q, right, a, b, c, d in lines:
    results = [
        score_qa(q, a),
        score_qa(q, b),
        score_qa(q, c),
        score_qa(q, d),
    ]

    best = results.index(max(results))
    best = chr(ord('A') + best)
    preds.append(best)
    scores.append(results)
    
    #print q, best, right
    
    if best == right:
        correct += 1

accuracy = float(correct) / len(lines)
print "Correct: %d Total: %d Accuracy: %f" % (correct, len(lines), accuracy)

Correct: 1082 Total: 2500 Accuracy: 0.432800


In [68]:
with open("predictions_training.csv", "w") as f:
    writer = csv.writer(f, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for i in xrange(len(lines)):
        writer.writerow([lines[i][0], preds[i]] + scores[i])

In [69]:
with open("../data/validation_set.tsv") as f:
    reader = csv.reader(f, delimiter="\t", strict=True, quoting=csv.QUOTE_NONE)
    header = next(reader)  # ignore header
    is_train_set = (len(header) == 7)
    lines = list(reader)

In [71]:
preds = []
scores = []
for id, q, a, b, c, d in lines:
    results = [
        score_qa(q, a),
        score_qa(q, b),
        score_qa(q, c),
        score_qa(q, d),
    ]

    best = results.index(max(results))
    best = chr(ord('A') + best)
    preds.append(best)
    scores.append(results)
    
    #print q, best, right

In [72]:
with open("predictions_validation.csv", "w") as f:
    writer = csv.writer(f, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for i in xrange(len(lines)):
        writer.writerow([lines[i][0], preds[i]] + scores[i])